In [1]:
# import sys
# !{sys.executable} -m pip install sodapy socrata-py python-dotenv

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import pandas as pd
from dotenv import load_dotenv
import pathlib
import requests

In [3]:
def fetch_nyc_data(dataset_id, total_records=1000, limit=100, where_clause=None):
    """Fetch data from NYC Open Data API using pagination"""
    
    all_data = []
    offset = 0
    
    while len(all_data) < total_records:
        url = f"https://data.cityofnewyork.us/resource/{dataset_id}.json"
        params = {
            '$offset': offset,
            '$limit': limit,
            '$$app_token': os.getenv("SOCRATA_APP_TOKEN")
        }
        
        if where_clause:
            params['$where'] = where_clause
        
        print(f"Fetching records {offset} to {offset + limit}...")
        
        try:
            response = requests.get(url, params=params) #timeout=60
            response.raise_for_status()
            
            data = response.json()
            
            if not data:
                print("No more data available")
                break
                
            all_data.extend(data)
            print(f"Retrieved {len(data)} records (total: {len(all_data)})")
            
            if len(data) < limit:
                print("Reached end of dataset")
                break
                
            offset += limit
                
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")
            break
    
    return all_data

# Fetch the data with custom where clause
# for some reason "status IN ('Open', In Progress)..." is not working properly
raw_data_open = fetch_nyc_data(
    "erm2-nwe9", 
    #total_records=1000, 
    #limit=100,
    where_clause="status IN ('Open') AND created_date >= '2025-01-01T00:00:00'"
)

raw_data_inProgress = fetch_nyc_data(
    "erm2-nwe9", 
    #total_records=1000, 
    #limit=100,
    where_clause="status IN ('In Progress') AND created_date >= '2025-01-01T00:00:00'"
)

# Combine
combined_data = raw_data_open + raw_data_inProgress

# Create DataFrame from combined data
results_df = pd.DataFrame(combined_data)

# Convert created_date to datetime just in case for proper sorting
results_df['created_date'] = pd.to_datetime(results_df['created_date'])

# Sort by created_date in descending order (most recent first)
results_df = results_df.sort_values('created_date', ascending=False).reset_index(drop=True)

columns_to_keep = [
    'unique_key',
    'created_date',
    'agency',
    'agency_name',
    'complaint_type',
    'descriptor',
    'incident_zip',
    'incident_address',
    'street_name',
    'address_type',
    'city',
    'facility_type',
    'status',
    'resolution_description',
    'resolution_action_updated_date',
    'community_board',
    'council_district',
    'bbl',
    'police_precinct',
    'borough',
    'park_borough',
    'x_coordinate_state_plane',
    'y_coordinate_state_plane',
    'latitude',
    'longitude',
    'location'
]

# Filter to only columns that exist in the dataframe
columns_to_keep = [col for col in columns_to_keep if col in results_df.columns]
results_df = results_df[columns_to_keep]

print(f"\nFinal DataFrame shape: {results_df.shape}")
print(f"\nStatus counts:\n{results_df['status'].value_counts()}")
print(f"\nDate range: {results_df['created_date'].min()} to {results_df['created_date'].max()}")

Fetching records 0 to 100...
Retrieved 100 records (total: 100)
Fetching records 100 to 200...
Retrieved 100 records (total: 200)
Fetching records 200 to 300...
Retrieved 100 records (total: 300)
Fetching records 300 to 400...
Retrieved 100 records (total: 400)
Fetching records 400 to 500...
Retrieved 100 records (total: 500)
Fetching records 500 to 600...
Retrieved 100 records (total: 600)
Fetching records 600 to 700...
Retrieved 100 records (total: 700)
Fetching records 700 to 800...
Retrieved 100 records (total: 800)
Fetching records 800 to 900...
Retrieved 100 records (total: 900)
Fetching records 900 to 1000...
Retrieved 100 records (total: 1000)
Fetching records 0 to 100...
Retrieved 100 records (total: 100)
Fetching records 100 to 200...
Retrieved 100 records (total: 200)
Fetching records 200 to 300...
Retrieved 100 records (total: 300)
Fetching records 300 to 400...
Retrieved 100 records (total: 400)
Fetching records 400 to 500...
Retrieved 100 records (total: 500)
Fetching rec

In [4]:
results_df.head()

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,address_type,...,council_district,bbl,police_precinct,borough,park_borough,x_coordinate_state_plane,y_coordinate_state_plane,latitude,longitude,location
0,67682467,2026-01-30 00:37:33,DOHMH,Department of Health and Mental Hygiene,Food Establishment,Food Worker Hygiene,11235,1717 AVENUE Z,AVENUE Z,ADDRESS,...,48,3074380024,Precinct 61,BROOKLYN,BROOKLYN,997526,153580,40.58820782754582,-73.95220089242231,"{'type': 'Point', 'coordinates': [-73.95220089..."
1,67695178,2026-01-29 22:08:00,DEP,Department of Environmental Protection,Water System,No Water (WNW),10308,234 WIMAN AVENUE,WIMAN AVENUE,ADDRESS,...,51,5053060069,Precinct 122,STATEN ISLAND,STATEN ISLAND,944067,135643,40.538893160760175,-74.14456877036254,"{'type': 'Point', 'coordinates': [-74.14456877..."
2,67683643,2026-01-29 13:03:52,HPD,Department of Housing Preservation and Develop...,DOOR/WINDOW,DOOR,10039,206 WEST 148 STREET,WEST 148 STREET,ADDRESS,...,09,1020330038,Precinct 32,MANHATTAN,MANHATTAN,1001331,239421,40.823814391623685,-73.93828356816849,"{'type': 'Point', 'coordinates': [-73.93828356..."
3,67682483,2026-01-29 12:44:20,DOHMH,Department of Health and Mental Hygiene,Non-Residential Heat,Inadequate or No Heat,10038,114 JOHN STREET,JOHN STREET,ADDRESS,...,01,1000690032,Precinct 1,MANHATTAN,MANHATTAN,982653,197089,40.70764039077776,-74.00576016400488,"{'type': 'Point', 'coordinates': [-74.00576016..."
4,67682482,2026-01-29 10:39:30,DOHMH,Department of Health and Mental Hygiene,Non-Residential Heat,Inadequate or No Heat,10022,805 3 AVENUE,3 AVENUE,ADDRESS,...,04,1013230047,Precinct 17,MANHATTAN,MANHATTAN,992290,214529,40.75550540171795,-73.97097996405773,"{'type': 'Point', 'coordinates': [-73.97097996..."


In [5]:
results_df.tail()

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,address_type,...,council_district,bbl,police_precinct,borough,park_borough,x_coordinate_state_plane,y_coordinate_state_plane,latitude,longitude,location
1995,64593560,2025-04-08 22:40:23,TLC,Taxi and Limousine Commission,For Hire Vehicle Complaint,Driver Complaint - Non Passenger,11215,392 3 AVENUE,3 AVENUE,ADDRESS,...,39,3009790005,Precinct 78,BROOKLYN,BROOKLYN,987251,184716,40.67367899270782,-73.98918130879443,"{'type': 'Point', 'coordinates': [-73.98918130..."
1996,64176137,2025-02-24 20:41:46,TLC,Taxi and Limousine Commission,Taxi Report,Driver Report - Passenger,10022,5 AVENUE,5 AVENUE,INTERSECTION,...,04,NaN,Precinct 19,MANHATTAN,MANHATTAN,991853,217951,40.76489830047662,-73.97255342667354,"{'type': 'Point', 'coordinates': [-73.97255342..."
1997,63938237,2025-01-30 17:33:02,TLC,Taxi and Limousine Commission,Taxi Report,Driver Report - Passenger,11369,LA GUARDIA AIRPORT,LA GUARDIA AIRPORT,UNRECOGNIZED,...,21,4009260001,Precinct 115,QUEENS,QUEENS,1018236,221443,40.77442086598845,-73.87729410513894,"{'type': 'Point', 'coordinates': [-73.87729410..."
1998,63818691,2025-01-20 19:05:52,TLC,Taxi and Limousine Commission,Taxi Report,Driver Report - Passenger,10036,1535 BROADWAY,BROADWAY,ADDRESS,...,03,1010177501,Precinct 18,MANHATTAN,MANHATTAN,988276,215534,40.75826660917717,-73.98546772308859,"{'type': 'Point', 'coordinates': [-73.98546772..."
1999,63757719,2025-01-13 13:35:42,TLC,Taxi and Limousine Commission,Taxi Report,Driver Report - Passenger,10013,BROOME STREET,BROOME STREET,INTERSECTION,...,01,NaN,Precinct 1,MANHATTAN,MANHATTAN,983424,202797,40.7233075908167,-74.00297997118642,"{'type': 'Point', 'coordinates': [-74.00297997..."


In [6]:
#results_df[:1000].status

In [7]:
# open_records = results_df[results_df['status'] == 'Open']
# print(open_records)